# Instant-ngp

This notebook aims to be a step-by-step guide to train NeRF models and rendering videos from them with nvidia's [instant-ngp](https://github.com/NVlabs/instant-ngp) software using:
 * **Colab** for the heavy lifting.
 * A low-resource **local computer** for the steps that require having a graphical user interface (GUI).

It has been tested on a GTX 1050ti in the local machine and an assigned Tesla T4 in the remote one.

Based on this [notebook](https://colab.research.google.com/drive/10TgQ4gyVejlHiinrmm5XOvQQmgVziK3i?usp=sharing) by [@myagues](https://github.com/NVlabs/instant-ngp/issues/6#issuecomment-1016397579), the main differences being the addition of steps 3 and 4 to ensure compatibility between the local machine and the models trained in the remote machine, of step 10 to render a video from the scene, and a more guided approach.

## 1.Connect to a GPU runtime

Connect your colab session to a GPU runtime and check that you have been assigned a GPU. It should have a minimum of 8GB of available memory.

In [1]:
!nvidia-smi

Mon May 19 17:07:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 2. Install dependencies and clone the instant-ngp repo

In [2]:
!apt update && apt install build-essential git python3-dev python3-pip libopenexr-dev libxi-dev libglfw3-dev libglew-dev libomp-dev libxinerama-dev libxcursor-dev colmap ffmpeg jq
!pip install --upgrade cmake

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,948 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,934 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://ppa.launchpadcontent.net/

In [3]:
!git clone --recursive https://github.com/nvlabs/instant-ngp
%cd instant-ngp

Cloning into 'instant-ngp'...
remote: Enumerating objects: 4372, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 4372 (delta 16), reused 11 (delta 11), pack-reused 4337 (from 2)
Receiving objects: 100% (4372/4372), 187.06 MiB | 30.93 MiB/s, done.
Resolving deltas: 100% (2789/2789), done.
Submodule 'dependencies/OpenXR-SDK' (https://github.com/KhronosGroup/OpenXR-SDK.git) registered for path 'dependencies/OpenXR-SDK'
Submodule 'dependencies/args' (https://github.com/Taywee/args) registered for path 'dependencies/args'
Submodule 'dependencies/dlss' (https://github.com/NVIDIA/DLSS) registered for path 'dependencies/dlss'
Submodule 'dependencies/glfw' (https://github.com/Tom94/glfw) registered for path 'dependencies/glfw'
Submodule 'dependencies/imgui' (https://github.com/ocornut/imgui.git) registered for path 'dependencies/imgui'
Submodule 'dependencies/pybind11' (https://github.com/Tom94/pybind11) registered for path 'dep

## 3. Set compute capability
Find the compute capability of the GPU in your **local** machine in the following link:
https://developer.nvidia.com/cuda-gpus

You need this to be able to open your trained models in `testbed` inside your local machine later on, so you can explore them or trace a camera path in order to generate a video from your scene.

In [4]:
compute_capability = "75" #@param [50, 52, 60, 61, 70, 72, 75, 80, 86, 87]
%env TCNN_CUDA_ARCHITECTURES=$compute_capability


env: TCNN_CUDA_ARCHITECTURES=75


## 4. Set the right network configuration
For compatibility between the model trained here and the local machine, a network with FP32 or FP16 is chosen.

https://docs.nvidia.com/deeplearning/tensorrt/support-matrix/index.html#hardware-precision-matrix

In [5]:
network_type = "FullyFusedMLP" if int(compute_capability) >= 70 else "CutlassMLP"
print(f"Using {network_type}")
%env NN_CONFIG_PATH = ./configs/nerf/base.json
!jq '.network.otype = "CutlassMLP" | .rgb_network.otype = "CutlassMLP"' $NN_CONFIG_PATH | sponge $NN_CONFIG_PATH

Using FullyFusedMLP
env: NN_CONFIG_PATH=./configs/nerf/base.json


## 5. Build the project and install python requirements

In [6]:
!cmake . -B build -DNGP_BUILD_WITH_GUI=OFF -DCMAKE_POLICY_VERSION_MINIMUM=3.5

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- The CUDA compiler identification is NVIDIA 12.5.82 with host compiler GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- No release type specified. Setting to 'Release'.
-- Obtained CUDA architectures from environment variable TCNN_CUDA_ARCHITECTURES=75
-- Targeting CUDA architectures: 75
-- Module su

In [7]:
!cmake --build build --config RelWithDebInfo -j `nproc`

[  5%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/pathescape.ptx
[  5%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/format.cc.o
[  7%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raystab.ptx
[ 10%] Building CUDA object CMakeFiles/optix_program.dir/src/optix/raytrace.ptx
[ 10%] Built target optix_program
[ 13%] Building CXX object dependencies/tiny-cuda-nn/dependencies/fmt/CMakeFiles/fmt.dir/src/os.cc.o
[ 15%] Linking CXX static library libfmt.a
[ 15%] Built target fmt
[ 21%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/common_host.cu.o
[ 21%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cpp_api.cu.o
[ 23%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/cutlass_mlp.cu.o
[ 26%] Building CUDA object dependencies/tiny-cuda-nn/CMakeFiles/tiny-cuda-nn.dir/src/encoding.cu.o
[ 28%] Building CUDA object dependencies/tiny-c

In [8]:
!pip3 install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.2/276.2 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 21.1 MB/s eta 0:00:00
  Created wheel for commentjson: filename=commentjson-0.9.0-py3-none-any.whl size=12077 sha256=c44f7c3fad30b2aec16ad1d7d547a8235fe292592ed3cf7dbedf432e64b1434c
  Stored in directory: /root/.cache/pip/wheels/f2/cb/58/77f1f0b3f5fa6eb3335cb0cb73de4d4bef93489dc3a3373ae8
  Created wheel for lark-parser: filename=lark_parser-0.7.8-py2.py3-none-any.whl size=62511 sha256=3b7ebd2857bdf3e2963d20495e02a54828960016a8aeac2589d1fa0b5b39b7f3
  Stored in directory: /root/.cache/pip/wheels/6c/c7/01/d277a7a673ffc81a1e71e816c2e5bc5f1602f9e8192c25d21c
Successfully built commentjson lark-parser


## 6. [LOCAL MACHINE] Run COLMAP on your scene
COLMAP doesn't work on machines without a GUI.

Go to your local machine and follow the [instructions](https://github.com/NVlabs/instant-ngp/blob/master/docs/nerf_dataset_tips.md#preparing-new-nerf-datasets) to run COLMAP from a video or a set of images to generate camera positions from your scene.

After this, you should have an images folder, with the images of your scene, and a `transforms.json` file with the camera information extracted by COLMAP.

## 7. Upload your scene

Mount your google drive

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Then upload the `images` folder and the output of COLMAP, `transforms.json`, to your drive. The structure should be similar to the following:
```
/content/drive/MyDrive/nerf_scenes/
└── fox
    ├── images
    │   ├── 00001.jpg
    │   └── 00002.jpg
    └── transforms.json
```



Enter the path to your scene

In [11]:
import os
scene_path = "/content/drive/MyDrive/nerf_studio/sewer" #@param {type:"string"}
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)

## 8. Train a model on your scene!

In [12]:
train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(scene_path, f"{train_steps}.ingp")
!python ./scripts/run.py {scene_path} --n_steps {train_steps} --save_snapshot {snapshot_path}

17:33:53 SUCCESS  Initialized CUDA 12.5. Active GPU is #0: Tesla T4 [75]
17:33:53 INFO     Loading NeRF dataset from
17:33:53 INFO       /content/drive/MyDrive/nerf_studio/sewer/transforms.json
17:33:54 PROGRESS []   0% (0/9)  0s/inf17:33:54 PROGRESS []  11% (1/9) 0s/4s17:33:54 PROGRESS []  22% (2/9) 0s/2s17:33:54 PROGRESS []  33% (3/9) 0s/2s17:33:55 PROGRESS []  44% (4/9) 0s/1s17:33:55 PROGRESS []  56% (5/9) 1s/2s17:33:55 PROGRESS []  67% (6/9) 1s/2s17:33:55 PROGRESS []  78% (7/9) 1s/2s17:33:56 PROGRESS []  89% (8/9) 2s/2s17:33:56 PROGRESS [] 100% (9/9) 2s/2s17:33:56 SUCCESS  Loaded 9 images after 2s
17:33:56 INFO       cam_aabb=[min=[1.69124,0.661147,0.103422], max=[1.84669,0.790244,0.849209]]
Training:   0% 0/2000 [00:00<?, ?steps/s]17:33:56 INFO     Loading network config from: /content/instant-ngp/configs/nerf/base.json
17:33:56 INFO     GridEncoding:  Nmin=16 b=2.97199 F=4 T=2^19 L=8
17:33:56 INFO     Density model: 3--[HashGrid]-->32--[CutlassMLP(neurons=64,layers=3)]-->1
17:33:

## 9. [LOCAL MACHINE] Generate a camera path

Congrats! You now have a trained nerf checkpoint. Now, in order to generate a video with it, you will need to open it in your local machine with `testbed` and generate a `base_cam.jon` file following these [instructions](https://github.com/NVlabs/instant-ngp#testbed-controls). Remember to launch with the `--no-train` argument so that it doesn't start to train on your PC. Setting up the cameras can make your GUI pretty laggy, you can try to play with the `--height` and `--width` parameters or cropping your scene with the `Crop aabb` options to optimize the performance.

Example command:
```
./build/instant-ngp /data/nerf/fox/2000.ingp
```

After you're done, **upload `base_cam.json` to the root folder of your scene.**

## 10. Render video

Make sure `base_cam.json` exists:

In [ ]:
video_camera_path = os.path.join(scene_path, "base_cam.json")
if not os.path.isfile(video_camera_path):
  raise FileNotFoundError(video_camera_path)

FileNotFoundError: /content/drive/MyDrive/nerf_studio/sewer/base_cam.json

Render the video

In [ ]:
video_n_seconds = 5 #@param {type:"integer"}
video_fps = 25 #@param {type:"integer"}
width = 720 #@param {type:"integer"}
height = 720 #@param {type:"integer"}
output_video_path = os.path.join(scene_path, "output_video.mp4")

!python scripts/run.py {snapshot_path} --video_camera_path {video_camera_path} --video_n_seconds 2 --video_fps 25 --width 720 --height 720 --video_output {output_video_path}
print(f"Generated video saved to:\n{output_video_path}")